In [1]:
%pip install numpy datasets Pillow

Note: you may need to restart the kernel to use updated packages.


# Implementation

In [2]:
import numpy as np

In [3]:
def softmax(x):
    x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return x / np.sum(x, axis=-1, keepdims=True)

def log_softmax(x):
    x = x - np.max(x, axis=-1, keepdims=True)
    return x - np.log(np.sum(np.exp(x), axis=-1, keepdims=True))

In [4]:
def loss_fn(logits, y):
    return -log_softmax(logits)[y]

def d_loss_fn(logits, y):
    p = softmax(logits)
    p[y] -= 1
    return p

In [5]:
def relu(x):
    return np.maximum(0, x)

def d_relu(x):
    return np.where(x > 0, 1, 0)

In [6]:
def init_params(in_size: int, out_size: int, hidden_sizes: list[int]):
    dims = [in_size] + hidden_sizes + [out_size]
    params = []
    for nin, nout in zip(dims[:-1], dims[1:]):
        # xavier uniform initialization for weight matrixes
        a = (6 / (nin + nout)) ** 0.5
        w = np.random.uniform(-a, a, size=[nin, nout])
        b = np.zeros(nout)
        params.append([w, b])
    return params

def forward(x, params):
    tape = [(None, x)]
    for w, b in params:
        z = x @ w + b
        x = relu(z)
        tape.append((z, x))
    return z, tape[:-1]

def backprop(x, y, params):
    logits, tape = forward(x, params)

    grad = []
    error = d_loss_fn(logits, y)
    for (z, a), (w, _) in zip(reversed(tape), reversed(params)):
        grad.append((error * a.reshape(-1, 1), error))
        if z is not None:
            error = error @ w.T
            error = error * d_relu(z)
    grad = list(reversed(grad))
    
    return grad

# Training on MNIST

In [7]:
def train_mnist():
    import datasets

    mnist = datasets.load_dataset("mnist")
    xtrain, ytrain = np.array(mnist["train"]["image"]).reshape(-1, 784) / 255.0, mnist["train"]["label"]
    xtest, ytest = np.array(mnist["test"]["image"]).reshape(-1, 784) / 255.0, mnist["test"]["label"]

    def compute_val_acc(params):
        val_correct = 0
        for x, y in zip(xtest, ytest):
            z, _ = forward(x, params)
            val_correct += np.argmax(z) == y
        return val_correct / len(xtest)

    params = init_params(784, 20, [32, 16])
    lr = 1e-3
    n_examples = 10000
    log_every_n_steps = 100

    for step in range(n_examples):
        # get random training example
        i = np.random.randint(len(xtrain))
        x, y = xtrain[i], ytrain[i]

        # compute gradient
        grad = backprop(x, y, params)

        # update the parameters
        for k in range(len(params)):
            params[k][0] -= lr * grad[k][0] 
            params[k][1] -= lr * grad[k][1]

        # log
        if step % log_every_n_steps == 0:
            print(f"step: {step} | acc: {compute_val_acc(params):.4f}")

train_mnist()

/Users/jay/code/backpropagation/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


step: 0 | acc: 0.0457
step: 100 | acc: 0.0761
step: 200 | acc: 0.0822
step: 300 | acc: 0.1149
step: 400 | acc: 0.1443
step: 500 | acc: 0.1772
step: 600 | acc: 0.1966
step: 700 | acc: 0.2349
step: 800 | acc: 0.2684
step: 900 | acc: 0.2979
step: 1000 | acc: 0.3200
step: 1100 | acc: 0.3586
step: 1200 | acc: 0.3876
step: 1300 | acc: 0.3999
step: 1400 | acc: 0.4189
step: 1500 | acc: 0.4233
step: 1600 | acc: 0.4278
step: 1700 | acc: 0.4469
step: 1800 | acc: 0.4444
step: 1900 | acc: 0.4570
step: 2000 | acc: 0.4629
step: 2100 | acc: 0.4896
step: 2200 | acc: 0.5072
step: 2300 | acc: 0.5123
step: 2400 | acc: 0.5087
step: 2500 | acc: 0.5209
step: 2600 | acc: 0.5285
step: 2700 | acc: 0.5401
step: 2800 | acc: 0.5468
step: 2900 | acc: 0.5605
step: 3000 | acc: 0.5795
step: 3100 | acc: 0.5805
step: 3200 | acc: 0.6000
step: 3300 | acc: 0.5831
step: 3400 | acc: 0.6299
step: 3500 | acc: 0.6398
step: 3600 | acc: 0.5819
step: 3700 | acc: 0.6111
step: 3800 | acc: 0.6303
step: 3900 | acc: 0.6369
step: 4000 |

# Verifying Backprop

We can verify that our backprop gives a correct by comparing it with a numerically computed approximation of the gradient, which we can obtain via (using a small value for $h$):

$$
f'(x) = \lim_{h\rightarrow 0} \frac{f(x - h) - f(x + h)}{2h}
$$

In [8]:
import copy

def numerical_grad(x, y, params, h=1e-6):
    compute_loss = lambda params: loss_fn(forward(x, params)[0], y)  # noqa: E731

    grad = copy.deepcopy(params)
    for i in range(len(params)):
        for j in range(len(params[i])):
            for k in np.ndindex(params[i][j].shape):
                prev_value = params[i][j][k]
                params[i][j][k] += h
                l = compute_loss(params)
                params[i][j][k] -= 2*h
                r = compute_loss(params)
                grad[i][j][k] = (l - r) / (2 * h)
                params[i][j][k] = prev_value

    return grad

def check_gradients():
    x, y = np.random.randn(256), np.random.randint(4)

    params = init_params(256, 4, [128, 64, 32, 16, 8])
    agrad = backprop(x, y, params)
    ngrad = numerical_grad(x, y, params)

    diff = []
    rels = []
    for (aw, ab), (bw, bb) in zip(agrad, ngrad):
        diff.append(np.linalg.norm(aw - bw) / (np.linalg.norm(aw) + np.linalg.norm(bw)))
        diff.append(np.linalg.norm(ab - bb) / (np.linalg.norm(ab) + np.linalg.norm(bb)))
        
        rels += (abs(aw - bw) / (abs(aw) + np.finfo(aw.dtype).smallest_subnormal)).flatten().tolist()
        rels += (abs(ab - bb) / (abs(bb) + np.finfo(aw.dtype).smallest_subnormal)).flatten().tolist()
    
    diff = np.array(diff)
    rels = np.array(rels)
    
    print("Max Difference Norm:", diff.max())
    print("Mean Difference Norm:", diff.mean())
    print()
    print("Max Relative Diff:", rels.max())
    print("Mean Relative Diff:", rels.mean())
    print()

    assert diff.max() < 1e-6
    assert rels.max() < 0.05
    print("gradient checking passed ✅")
    print()

check_gradients()

Max Difference Norm: 4.874888229837376e-09
Mean Difference Norm: 1.989803113430697e-09

Max Relative Diff: 0.00035643038229495977
Mean Relative Diff: 7.079203981904641e-08

gradient checking passed ✅

